# Trading Report Card : Your Tear Sheet

## Select Environment & Start Date

In [ ]:
env = "PAPER" # PAPER / PROD / BACKTEST
start_date = "2021-01-01"

#### Imports

In [ ]:
# silence warnings
import warnings
from datetime import datetime
import liualgotrader.analytics.analysis as ana
from liualgotrader.models.gain_loss import GainLoss, TradeAnalysis
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
from empyrical import roll_max_drawdown
from scipy.stats import kurtosis, skew
from IPython.display import HTML, display, Markdown
%matplotlib inline
warnings.filterwarnings("ignore")


nest_asyncio.apply()

#### Load Portfolio data from database

In [ ]:
returns, investments, percentages = ana.portfolio_return(env, start_date)
gl_df = await GainLoss.load(env, datetime.strptime(start_date, "%Y-%m-%d"))
print(f"loaded {len(gl_df)} gain_loss entries")
ta_df = await TradeAnalysis.load(env,  datetime.strptime(start_date, "%Y-%m-%d"))
print(f"loaded {len(ta_df)} trade_analysis entries")

In [ ]:
display(Markdown("## $ Profile / Loss"))
display(returns)
display(Markdown("## % Profile / Loss"))
display(percentages)
display(Markdown("## Total $ investments"))
display(investments)


## Revenue & Percentage per strategy

In [ ]:
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
with plt.xkcd():
    for column in columns:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))  # 1 row, 2 columns
        ax1.set_ylabel(f"{column} $")
        ax2.set_ylabel(f"{column} %")
        plt.xticks(rotation=45)
        returns[column].plot(ax=ax1)
        percentages[column].plot(ax=ax2, color="red")
        plt.tight_layout()

## Strategy/Symbol affinity to Revenue & Percentage

In [ ]:
counts = gl_df.symbol.value_counts().rename("c").to_frame()
display(gl_df[gl_df.symbol.isin(counts.loc[counts.c >= 3].index.tolist())])

## Accumulative Revenue & Draw-down

In [ ]:
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
for column in columns:
    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(10, 4), gridspec_kw={"height_ratios": [3, 1]}
    )  
    ax1.set_ylabel(f"Accumulative\n{column} $")
    plt.xticks(rotation=45)
    returns[column].cumsum().plot(ax=ax1)
    roll_max_drawdown(returns[column].cumsum(), window=1).plot(ax=ax2)
    ax2.set_ylabel("drawdown")
    plt.tight_layout()

## Strategy performance distribution by revenue

In [ ]:
df = pd.DataFrame()
df["strategy"] = returns.columns.unique()
df["mean"] = df.strategy.apply(lambda x: returns[x].dropna().mean())
df["std"] = df.strategy.apply(lambda x: returns[x].dropna().std())
df["skew"] = df.strategy.apply(lambda x: skew(returns[x].dropna()))
df["kurtosis"] = df.strategy.apply(lambda x: kurtosis(returns[x].dropna()))
display(Markdown("### Summary"))
display(df)
display(Markdown("### Revenue Histogram per strategy"))
columns = list(returns.columns)
columns.insert(0, columns.pop(columns.index("revenue")))
with plt.xkcd():
    for column in columns:
        returns[column].hist()
        plt.xticks(rotation=45)
        plt.ylabel(column)
        plt.xlabel('$ Revenue')
        plt.show()

## Strategy performance by `R Units`

In [ ]:
counts = ta_df.symbol.value_counts().rename('c').to_frame()
with plt.xkcd():
    for strat in ta_df.algo_name.unique().tolist():
        ta_df[
            (ta_df.symbol.isin(counts.loc[counts.c > 0].index.tolist()))
            & (ta_df.algo_name == strat)
        ].r_units.hist(bins=10, density=True)
        plt.xticks(rotation=45)
        plt.ylabel(strat)
        plt.xlabel('r units')
        plt.show()